## Dataset exploration

This is probably going to be a minimal EDA notebook looking at tool calling datasets that exists in market right now

In [22]:
#!pip install datasets

In [2]:
from datasets import load_dataset

### Hermes-Function-Calling v1

- Apache 2.0
- Single-Turn: 2k
- Func_calling: 2k
- Glaive: 5k
- Json mode agent: 1.3k
- Json mode single: 1.24k

https://huggingface.co/datasets/NousResearch/hermes-function-calling-v1

In [27]:
d = load_dataset("NousResearch/hermes-function-calling-v1")

In [32]:
d['train'][123].keys()

dict_keys(['id', 'conversations', 'category', 'subcategory', 'task'])

In [30]:
d['train'][123]

{'id': '936b90da-dbeb-4864-a6e6-28899965265d',
 'conversations': [{'from': 'system',
   'value': "You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.\n<tools>\n[{'type': 'function', 'function': {'name': 'scale_up_cluster', 'description': 'Scales up the number of compute nodes in an HPC cluster.', 'parameters': {'type': 'object', 'properties': {'cluster_name': {'type': 'string', 'description': 'The name of the HPC cluster to scale up.'}, 'node_type': {'type': 'string', 'description': 'The type of compute node to add to the cluster.'}, 'additional_nodes': {'type': 'integer', 'description': 'The number of additional compute nodes to add to the cluster.'}}, 'required': ['cluster_name', 'node_type', 'additional_nodes']}}}, {'type': 'function', 'function': {'name': 'scale_down_cluster', 'description': '

In [32]:
d['train']['category']

['IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'IoT and Home Automation',
 'Quantum Computing',
 'Quantum Computing',
 'Quantum Computing',
 'Quantum Computing',
 'Quantum Computing',
 'Quantum Computing',
 'Quantum Computing',
 'Quantum Computing',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 'Services Industry Software',
 

### Pandora Tool Calling

- Based on glaive-ai function calling
- BSD-3
- 100k examples
- 
https://huggingface.co/datasets/danilopeixoto/pandora-tool-calling

In [35]:
d = load_dataset("danilopeixoto/pandora-tool-calling")

README.md:   0%|          | 0.00/607 [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/84.5M [00:00<?, ?B/s]

valid.parquet:   0%|          | 0.00/4.54M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/4.64M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/101114 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5618 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5619 [00:00<?, ? examples/s]

In [38]:
d['train'][21313]

{'messages': [{'role': 'system',
   'content': 'You are a helpful assistant with access to functions and tools. Use them if required.',
   'tools': None,
   'tool_calls': None,
   'name': None},
  {'role': 'system',
   'content': None,
   'tools': '[{"name": "generate_random_quote", "description": "Generate a random inspirational quote", "parameters": {}}, {"name": "calculate_loan_payment", "description": "Calculate the monthly payment for a loan", "parameters": {"type": "object", "properties": {"loan_amount": {"type": "number", "description": "The total loan amount"}, "interest_rate": {"type": "number", "description": "The annual interest rate on the loan"}, "loan_term": {"type": "integer", "description": "The duration of the loan in months"}}, "required": ["loan_amount", "interest_rate", "loan_term"]}}]',
   'tool_calls': None,
   'name': None},
  {'role': 'user',
   'content': "I'm feeling a bit down today. Can you share something inspiring?",
   'tools': None,
   'tool_calls': None

### Reflection-v1 Glaive ai

- Apache 2.0
- 60k examples
- https://huggingface.co/datasets/glaiveai/reflection-v1

In [51]:
d = load_dataset("glaiveai/glaive-function-calling-v2")

In [52]:
d['train'][1455]

{'system': 'SYSTEM: You are a helpful assistant, with no access to external functions.\n\n',
 'chat': 'USER: How can I write a C# program that calculates the difference between two given dates in years?\nHere\'s a starter code:\nDateTime givenDate = new DateTime(2015, 8, 12);\nDateTime currentDate = DateTime.Today;\nTimeSpan difference = currentDate - givenDate;\nint years = (int)(difference.TotalDays / 365.25);\nConsole.WriteLine("The difference between the given date and today\'s date is: " + years + " years.");\nYou can replace the given date with any date of your choice. How can this code be optimized for better performance?\n\nASSISTANT: The code you provided is already quite efficient, but there are a few optimizations you can make:\n1. Instead of dividing by 365.25 to get the number of years, you can use the TotalDays property of the TimeSpan object and divide by the average number of days in a year (365.2425). This will give you a more accurate result.\n2. You can use the TryPa

### Glaive-Function-calling-v2

- Apache 2.0
- 113k rows
- https://huggingface.co/datasets/glaiveai/glaive-function-calling-v2

### Glaive-Function-calling-v1

- Apache 2.0
- 52k rows
- https://huggingface.co/datasets/glaiveai/glaive-function-calling

In [3]:
d = load_dataset("glaiveai/glaive-function-calling")

In [4]:
d['train'][12312]

{'sample': 'SYSTEM: You are an helpful assistant who has access to the following functions to help the user, you can use the functions if needed-\n{\n    "name": "calculate_age",\n    "description": "Calculate the age based on the birthdate",\n    "parameters": {\n        "type": "string",\n        "description": "The birthdate in YYYY-MM-DD format",\n        "required": true\n    }\n}\nUSER: Can you help me calculate my age?\nASSISTANT: Sure! I can help you with that. Please provide me with your birthdate in YYYY-MM-DD format.\nUSER: My birthdate is 1990-05-15.\nASSISTANT: <functioncall> {"name": "calculate_age", "arguments": \'{"birthdate": "1990-05-15"}\'}\nFUNCTION RESPONSE: {"age": 31}\nASSISTANT: Based on the provided birthdate, your age is 31.\nUSER: Thank you for calculating my age!\nASSISTANT: You\'re welcome! If you have any more questions or need further assistance, feel free to ask.'}

### Salesforce-xlam-functioncalling-60k

- CC-by-4
- 60k rows
- https://huggingface.co/datasets/Salesforce/xlam-function-calling-60k

In [5]:
d = load_dataset("Salesforce/xlam-function-calling-60k")

In [6]:
d['train'][555]

{'id': 555,
 'query': 'Determine the area of a triangle with a base of 7.5 cm and a height of 3.2 cm.',
 'answers': '[{"name": "triangle_area", "arguments": {"base": 7.5, "height": 3.2}}]',
 'tools': '[{"name": "triangle_area", "description": "Computes the area of a triangle given its base and height.", "parameters": {"base": {"description": "The length of the base of the triangle.", "type": "float"}, "height": {"description": "The height of the triangle.", "type": "float"}}}, {"name": "batting_average", "description": "Calculates the batting average of a baseball player based on the number of hits and at-bats.", "parameters": {"num_hits": {"description": "The number of hits.", "type": "int", "default": 3}, "num_at_bats": {"description": "The number of at-bats.", "type": "int", "default": 3}, "precision": {"description": "The number of decimal places to round the batting average. Defaults to 3.", "type": "int, optional"}}}, {"name": "fibonacci_numbers", "description": "Generates the fi

### Argilla-Tool-Calling

- Apache 2.0
- 60k examples
- https://huggingface.co/datasets/argilla-warehouse/apigen-synth-trl

In [33]:
d = load_dataset("argilla-warehouse/apigen-synth-trl")

In [34]:
d['train'][43123]

{'messages': [{'content': 'You are an expert in composing functions. You are given a question and a set of possible functions. \nBased on the question, you will need to make one or more function/tool calls to achieve the purpose. \nIf none of the functions can be used, point it out and refuse to answer. \nIf the given question lacks the parameters required by the function, also point it out.\n\nThe output MUST strictly adhere to the following format, and NO other text MUST be included.\nThe example format is as follows. Please make sure the parameter type is correct. If no function call is needed, please make the tool calls an empty list \'[]\'.\n```\n<tool_call>[\n{"name": "func_name1", "arguments": {"argument1": "value1", "argument2": "value2"}},\n... (more tool calls as required)\n]</tool_call>\n```',
   'role': 'system'},
  {'content': 'You have access to the following tools:\n<tools>[{"type":"function","function":{"name":"remove_dir_recursively","description":"Recursively removes 

In [1]:
import transformers
import torch

model_id = "meta-llama/Llama-3.3-70B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cu",
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Device set to use cuda:0


In [13]:
EXAMPLE = d['train'][123]

In [14]:
EXAMPLE

{'id': 123,
 'query': 'Can you fetch the latest betting tips sorted by descending ID and also provide the schedule for the Los Angeles Lakers in the 2023 season?',
 'answers': '[{"name": "get_all_predictions", "arguments": {"sort": "-id"}}, {"name": "get_team_schedule", "arguments": {"season": "2023", "teamabv": "LAL"}}]',
 'tools': '[{"name": "get_all_predictions", "description": "Fetches all betting tips from the API with configurable parameters for connection, content type, sorting, and API key.", "parameters": {"connection": {"description": "Specifies the connection type for the API request. Default is \'keep-alive\'.", "type": "str", "default": "keep-alive"}, "content_type": {"description": "Sets the content type for the API request. Default is \'application/json\'.", "type": "str", "default": "application/json"}, "sort": {"description": "Criterion for sorting the betting tips. Default is \'-id\' for descending order.", "type": "str", "default": "-id"}}}, {"name": "get_team_schedu

In [16]:
SYSTEM_PROMPT = """
You are high 170IQ reasoning scientist model, your job is to take existing messages from conversations with a dumb AI and 
add reasoning to it, remember we are adding chain of thought to the existing example without modifying what it does, 
think out loud and max out your tokens. 

Below is an example: 

INPUT: "{'id': 12314,
 'query': 'Can you provide the innings details for the cricket event with ID 12345 and the football predictions for the Premier League on August 15, 2023?',
 'answers': '[{"name": "innings_cricket", "arguments": {"event_id": 12345}}, {"name": "daily_predictions", "arguments": {"league": "Premier League", "date": "2023-08-15"}}]',
 'tools': '[{"name": "innings_cricket", "description": "Fetch innings data for a cricket event given the event ID.", "parameters": {"event_id": {"description": "The unique identifier of the cricket event.", "type": "int", "default": "10188874"}}}, {"name": "daily_predictions", "description": "Fetches daily football predictions using various filters and pagination support.", "parameters": {"league": {"description": "The league identifier for filtering predictions. Example: \'1\'.", "type": "str, optional", "default": ""}, "page": {"description": "The page number for pagination. Example: \'1\'.", "type": "str, optional", "default": "1"}, "market": {"description": "The market type for filtering predictions. Example: \'1X2\', \'OU25\', \'bts\'.", "type": "str, optional", "default": ""}, "date": {"description": "The date for filtering predictions in the format \'YYYY-MM-DD\'. Example: \'2022-08-13\'.", "type": "str, optional", "default": ""}}}]'}"

OUTPUT: "{'id': 12314,
 'query': 'Can you provide the innings details for the cricket event with ID 12345 and the football predictions for the Premier League on August 15, 2023?',
 'reasoning' : 'To address the query regarding the innings details for a specific cricket event and the football predictions for the Premier League on a given date, we must break down the problem into its components and apply logical reasoning to each part. The query involves two distinct tasks:\n\n1. **Fetching Innings Details for a Cricket Event**: This task requires accessing data related to a cricket event specified by an ID (in this case, 12345). The tool designed for this purpose is `innings_cricket`, which takes an `event_id` as its argument. By providing the `event_id` as 12345 to the `innings_cricket` tool, we can obtain the innings details for the specified cricket event.\n\n2. **Fetching Football Predictions for the Premier League**: The second task involves obtaining football predictions for the Premier League on a specific date, August 15, 2023. The tool suited for this task is `daily_predictions`, which can filter predictions based on several parameters, including `league`, `date`, and others. To get the desired predictions, we need to set the `league` parameter to "Premier League" and the `date` parameter to "2023-08-15" when using the `daily_predictions` tool.\n\nGiven the structure of the query and the tools available, the logical approach to solving this problem involves:\n\n- **Identifying the Tools Needed**: Recognize that two separate tools (`innings_cricket` and `daily_predictions`) are required to fulfill the different parts of the query.\n- **Applying Parameters**: Understand the parameters each tool requires. For `innings_cricket`, the critical parameter is `event_id`. For `daily_predictions`, the key parameters are `league` and `date`.\n- **Executing the Tools**: With the parameters identified, execute each tool with its respective parameters. For `innings_cricket`, execute with `event_id = 12345`. For `daily_predictions`, execute with `league = "Premier League"` and `date = "2023-08-15"`.\n- **Combining the Results**: Finally, combine the results from both tools to provide a comprehensive answer to the query, ensuring that both the innings details for the cricket event and the football predictions for the Premier League on the specified date are included.\n\nThis step-by-step logical reasoning process ensures that the query is addressed in its entirety, leveraging the capabilities of the provided tools to fetch and combine the required information.'
 'answers': '[{"name": "innings_cricket", "arguments": {"event_id": 12345}}, {"name": "daily_predictions", "arguments": {"league": "Premier League", "date": "2023-08-15"}}]',
 'tools': '[{"name": "innings_cricket", "description": "Fetch innings data for a cricket event given the event ID.", "parameters": {"event_id": {"description": "The unique identifier of the cricket event.", "type": "int", "default": "10188874"}}}, {"name": "daily_predictions", "description": "Fetches daily football predictions using various filters and pagination support.", "parameters": {"league": {"description": "The league identifier for filtering predictions. Example: \'1\'.", "type": "str, optional", "default": ""}, "page": {"description": "The page number for pagination. Example: \'1\'.", "type": "str, optional", "default": "1"}, "market": {"description": "The market type for filtering predictions. Example: \'1X2\', \'OU25\', \'bts\'.", "type": "str, optional", "default": ""}, "date": {"description": "The date for filtering predictions in the format \'YYYY-MM-DD\'. Example: \'2022-08-13\'.", "type": "str, optional", "default": ""}}}]'}"


Make sure you follow this approach and return outputs that add the 'reasoning' traces but DO NOT alter any existing calls
"""

In [20]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": f"{EXAMPLE}"},
]

outputs = pipeline(
    messages,
    max_new_tokens=8126,
)
print(outputs[0]["generated_text"][-1])

KeyError: 'assistant'

In [25]:
print(outputs[0]["generated_text"][-1]['content'])

{'id': 123, 
 'query': 'Can you fetch the latest betting tips sorted by descending ID and also provide the schedule for the Los Angeles Lakers in the 2023 season?', 
'reasoning': 'To address this query, we must break it down into its components and apply logical reasoning to each part. The query involves two distinct tasks:

1. **Fetching the Latest Betting Tips Sorted by Descending ID**: This task requires accessing betting tips data and sorting it in descending order based on the ID. The tool designed for this purpose is `get_all_predictions`, which allows for configurable sorting. By setting the `sort` parameter to `-id`, we can obtain the betting tips sorted as required.

2. **Fetching the Schedule for the Los Angeles Lakers in the 2023 Season**: The second task involves obtaining the schedule for a specific NBA team, the Los Angeles Lakers, for the 2023 season. The tool suited for this task is `get_team_schedule`, which can fetch the schedule based on the season and team. To get t